<a href="https://colab.research.google.com/github/9645258/aiffel/blob/main/01_exploration/%5BE-11%5Dexploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🌲 EXPLORATION 11-1 :한국어 데이터로 챗봇 만들기**

## **🌿 01_데이터 준비**

### **🍃 01-1_필요 라이브러리 import**

- **os** (운영체제)  
- **re** (정규 표현식)  
- **numpy**  (수 표현)  
- **pandas** (데이터 프레임)  
- **matplotlib** (시각화)  
- **tensorflow** (딥러닝 프로그램 구현)  
- **tensorflow_datasets** (텐서플로 내장 데이터 셋 사용)  
SubwordTextEncoder를 토크나이저로 사용

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

### **🍃 01-2_필요 데이터 import**

- **구글 드라이브 마운트**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/aiffel/01-2_exploration/E-11/data

/content/drive/MyDrive/Colab Notebooks/aiffel/01-2_exploration/E-11/data


- **학습 데이터 import**

In [ ]:
chatbot_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aiffel/01-2_exploration/E-11/data/ChatbotData.csv')

### **🍃 01-3_데이터 확인**

- **데이터 확인**  
3개 열 (Q:질문, A:답변, label)  
전체 데이터: 11,823개  
질문 데이터: 11,823개  
답변 데이터: 11,823개

In [ ]:
chatbot_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [ ]:
print('전체 데이터 수:', len(chatbot_data))
print('질문(Q) 데이터 수:', len(chatbot_data['Q']))
print('답변(A) 데이터 수:', len(chatbot_data['A']))

전체 데이터 수: 11823
질문(Q) 데이터 수: 11823
답변(A) 데이터 수: 11823


## **🌿 02_데이터 전처리**

### **🍃 02-1_정규 표현식**


- **정규 표현식**  
    - **re.sub(정규 표현식 검색 패턴, 검색 대상 문자열, 변경하고 싶은 문자)**  
- **문자열 대소문자 변환**  
    - **upper()** : 소문자 → 대문자  
    - **lower()** : 대문자 → 소문자  
    - **capitalize()** : 맨 첫 글자만 대문자  
    - **title()** : 알파벳 외의 문자로 나눠져 있는 영단어들의 맨 첫글자만 대문자  
- **공백 제거**  
    - **strip()** : 양 옆 공백 제거  
    - **lstrip()** : 왼쪽 공백 제거  
    - **rstrip()** : 오른쪽 공백 제거  

In [ ]:
def preprocess_sentence(sentence):
    #영문 대문자 표기 (줄임말 사용이 많으므로 대문자 사용)
    sentence = sentence.upper().strip()

    # 단어와 특수기호 사이에 공백 생성
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # a-z, A-Z, 특수기호, 숫자, 한글을 제외한 모든 글자 공백으로 대체
    sentence = re.sub(r"[^a-zA-Z0-9가-힣ㄱ-ㅎㅏ-ㅣ?.!,]+", " ", sentence)
    sentence = sentence.strip()

    return sentence

### **🍃 02-2_질문/답변 데이터셋 구성 및 확인**

- **질문/답변 데이터셋 구성**

In [ ]:
def load_conversations(questions, answers):
    inputs, outputs = [], []
    
    for question, answer in zip(questions, answers):
        inputs.append(preprocess_sentence(question))
        outputs.append(preprocess_sentence(answer))
        
    return inputs, outputs

In [ ]:
questions, answers = load_conversations(chatbot_data['Q'], chatbot_data['A'])

- **질문/답변 데이터셋 수 확인**

In [ ]:
print('질문(Q) 데이터 수:', len(questions))
print('답변(A) 데이터 수:', len(answers))

질문(Q) 데이터 수: 11823
답변(A) 데이터 수: 11823


- **질문/답변 데이터셋 병렬 확인**  
특수기호가 공백으로 분리됨 / 본문 병렬 저장 확인

In [ ]:
print('전처리 후의 60번째 질문 샘플: {}'.format(questions[59]))
print('전처리 후의 60번째 답변 샘플: {}'.format(answers[59]))

전처리 후의 60번째 질문 샘플: 감정이 쓰레기통처럼 엉망진창이야
전처리 후의 60번째 답변 샘플: 자신을 더 사랑해주세요 .


## **🌿 03_토큰화**

### **🍃 03-1_토크나이저**

- **질문과 답변 데이터 셋을 활용한 단어장 생성**  
각 단어에 고유한 정수 인덱스 부여  

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size = 2**13)

### **🍃 03-2_시작, 끝 토큰 추가**

- **디코더의 문정 생성 과정에서 사용할 시작토큰과 종료토큰 임의로 단어장에 추가해 정수를 부여**  

In [ ]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

- **토큰 2개(시작, 종료)를 추가하므로 단어장의 크기에 2개 추가**

In [ ]:
VOCAB_SIZE = tokenizer.vocab_size + 2

### **🍃 03-3_정수 인코딩 / 패딩**

- **최대 길이 40으로 패딩 진행**

In [ ]:
MAX_LENGTH = 40

- **정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩 진행 함수**

In [ ]:
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    
    for (sentence1, sentence2) in zip(inputs, outputs):
        # 시작 토큰, 종료 토큰 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

        # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
  
  # 패딩: 최대 길이 - 40
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
    return tokenized_inputs, tokenized_outputs

In [ ]:
questions, answers = tokenize_and_filter(questions, answers)

In [ ]:
print('단어장 크기:', (VOCAB_SIZE))
print('토큰나이징 후 질문(Q) 샘플 개수: {}'.format(len(questions)))
print('토큰나이징 후 답변(A) 샘플 개수: {}'.format(len(answers)))

단어장 크기: 8172
토큰나이징 후 질문(Q) 샘플 개수: 11823
토큰나이징 후 답변(A) 샘플 개수: 11823


### **🍃 03-4_교사강요 사용**

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 11823

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

## **🌿 04_모델 구성**

### **🍃 04-1_포지셔널 인코딩**

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) /
                            tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

### **🍃 04-2_스케일드 닷 프로덕트 어텐션**

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    if mask is not None:
        logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

### **🍃 04-3_멀티 헤드 어텐션**

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V Dense를 적용
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위한 머리
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # Scaled Dot Product Attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에 Dense 한 번 더 적용
    outputs = self.dense(concat_attention)

    return outputs

### **🍃 04-4_패딩 마스킹**

In [ ]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]

### **🍃 04-5_룩 어헤드 마스킹**

In [ ]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - \
        tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

### **🍃 04-6_인코더 층 생성**

- **하나의 레이어 안에 두개의 서브 레이어 존재**

In [ ]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

### **🍃 04-7_인코더 층을 쌓아 인코더 생성**

In [ ]:
def encoder(vocab_size, num_layers, units, d_model, num_heads, dropout, name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

### **🍃 04-8_디코더 생성**

- **하나의 레이어 안에 세개의 서브 레이어 존재**

In [ ]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={'query': inputs, 'key': inputs, 'value': inputs, 'mask': look_ahead_mask})

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={'query': attention1, 'key': enc_outputs, 'value': enc_outputs, 'mask': padding_mask})

  # 마스크드 멀티 헤드 어텐션의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask], outputs=outputs, name=name)

### **🍃 04-9_디코더 층을 쌓아 디코더 생성**

In [ ]:
def decoder(vocab_size, num_layers, units, d_model, num_heads, dropout, name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask], outputs=outputs, name=name)

## **🌿 05_모델 정의 및 학습**

### **🍃 05-1_모델 정의**

In [ ]:
def transformer(vocab_size, num_layers, units, d_model, num_heads, dropout, name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None), name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(create_look_ahead_mask, output_shape=(1, None, None), name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None), name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, units=units, d_model=d_model, num_heads=num_heads, dropout=dropout,)(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, units=units, d_model=d_model, num_heads=num_heads, dropout=dropout,)(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

### **🍃 05-2_모델 생성**

In [ ]:
# Model 생성
tf.keras.backend.clear_session()

# 하이퍼파라미터

# 인코더와 디코더의 층의 개수
NUM_LAYERS = 2 
# 인코더와 디코더 내부의 입, 출력의 고정 차원
D_MODEL = 256 
# 멀티 헤드 어텐션에서의 헤드 수 
NUM_HEADS = 8 
# 피드 포워드 신경망의 은닉층의 크기
UNITS = 512 
# 드롭아웃의 비율
DROPOUT = 0.1

model = transformer(vocab_size=VOCAB_SIZE, num_layers=NUM_LAYERS, units=UNITS, d_model=D_MODEL, num_heads=NUM_HEADS, dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, None)]       0           []                               
                                                                                                  
 dec_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 enc_padding_mask (Lambda)      (None, 1, 1, None)   0           ['inputs[0][0]']                 
                                                                                                  
 encoder (Functional)           (None, None, 256)    3146240     ['inputs[0][0]',                 
                                                                  'enc_padding_mask[0][0

### **🍃 05-3_손실 함수**

In [ ]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

 ### **🍃 05-4_커스텀된 학습률**

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

 ### **🍃 05-5_정확도**

In [ ]:
def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

 ### **🍃 05-6_모델 예측 함수 제작**

In [ ]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)

def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))
  print('\n')

  return predicted_sentence

 ### **🍃 05-7_모델 컴파일**

In [ ]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

 ### **🍃 05-8_모델 학습**

In [ ]:
EPOCHS = 70
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/70
185/185 [==============================] - 60s 288ms/step - loss: 1.4515 - accuracy: 0.0222
Epoch 2/70
185/185 [==============================] - 53s 289ms/step - loss: 1.1769 - accuracy: 0.0495
Epoch 3/70
185/185 [==============================] - 53s 287ms/step - loss: 1.0042 - accuracy: 0.0507
Epoch 4/70
185/185 [==============================] - 53s 289ms/step - loss: 0.9281 - accuracy: 0.0547
Epoch 5/70
185/185 [==============================] - 53s 286ms/step - loss: 0.8705 - accuracy: 0.0576
Epoch 6/70
185/185 [==============================] - 53s 287ms/step - loss: 0.8125 - accuracy: 0.0613
Epoch 7/70
185/185 [==============================] - 53s 286ms/step - loss: 0.7489 - accuracy: 0.0668
Epoch 8/70
185/185 [==============================] - 53s 284ms/step - loss: 0.6776 - accuracy: 0.0744
Epoch 9/70
185/185 [==============================] - 53s 286ms/step - loss: 0.5983 - accuracy: 0.0833
Epoch 10/70
185/185 [==============================] - 53s 287ms/step - l

## **🌿 06_모델 평가**

In [ ]:
sentence_generation("훌쩍 여행을 떠나고 싶어.")

입력 : 훌쩍 여행을 떠나고 싶어.
출력 : 이김에 떠나보세요 .




'이김에 떠나보세요 .'

In [ ]:
sentence_generation("봄이 가고 여름이 오고 있어.")

입력 : 봄이 가고 여름이 오고 있어.
출력 : 구릿빛 피부 좋죠 !




'구릿빛 피부 좋죠 !'

In [ ]:
sentence_generation("오늘 너무 지치고 힘들어서 쉬고 싶어.")

입력 : 오늘 너무 지치고 힘들어서 쉬고 싶어.
출력 : 아무 생각 하지 말고 쉬세요 .




'아무 생각 하지 말고 쉬세요 .'

In [ ]:
sentence_generation("눈을 감았는데 생각이 많아서 잠이 안와.")

입력 : 눈을 감았는데 생각이 많아서 잠이 안와.
출력 : 끝낼 때를 아는 것도 중요한 것 같아요 .




'끝낼 때를 아는 것도 중요한 것 같아요 .'

In [ ]:
sentence_generation("밥은 제대로 챙겨 먹어야지!")

입력 : 밥은 제대로 챙겨 먹어야지!
출력 : 건강 생각해서 챙겨드세요 .




'건강 생각해서 챙겨드세요 .'

In [ ]:
sentence_generation("계속 검색을 해도 나오지 않는 걸!")

입력 : 계속 검색을 해도 나오지 않는 걸!
출력 : 간절한 만큼 할 수 있을 거예요 .




'간절한 만큼 할 수 있을 거예요 .'

In [ ]:
sentence_generation("언제나 새로운 날들은 두근거려!")

입력 : 언제나 새로운 날들은 두근거려!
출력 : 저도 두근거리네요 .




'저도 두근거리네요 .'

In [ ]:
sentence_generation("내 마음에도 곰팡이가 핀 것 같아!")

입력 : 내 마음에도 곰팡이가 핀 것 같아!
출력 : 지금은 괜찮길 바랄게요 .




'지금은 괜찮길 바랄게요 .'

In [ ]:
sentence_generation("언제 나는 어른이 될까?")

입력 : 언제 나는 어른이 될까?
출력 : 물리적 나이가 아니라 정신적 나이가 중요하니까요 .




'물리적 나이가 아니라 정신적 나이가 중요하니까요 .'

In [ ]:
sentence_generation("오늘 아침 커피 한 잔 어때?")

입력 : 오늘 아침 커피 한 잔 어때?
출력 : 건강에 좋은 습관이네요 .




'건강에 좋은 습관이네요 .'

In [ ]:
sentence_generation("내 행복은 어디에 있을까?")

입력 : 내 행복은 어디에 있을까?
출력 : 있는 그대로의 나로 살아야 행복하게 살 수 있어요 .




'있는 그대로의 나로 살아야 행복하게 살 수 있어요 .'

In [ ]:
sentence_generation("내가 뭘 잘못한 게 있니?")

입력 : 내가 뭘 잘못한 게 있니?
출력 : 상황이 그렇게 만든 거예요 .




'상황이 그렇게 만든 거예요 .'

<hr>